# Download Data

> Module to download the dataset

In [ ]:
# | default_exp preprocess.create_annotations

In [ ]:
# | hide
%reload_ext nb_black
%reload_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=

from nbdev.showdoc import *
import sys

__root = "../../"
sys.path.append(__root)

In [ ]:
# | export
from clip_video_classifier.cli import cli
from torch_snippets import *
import requests
from pytorchvideo.data.encoded_video import EncodedVideo

In [ ]:
# | export


def wget(url, to):
    response = requests.get(url)
    makedir(parent(to))
    with open(to, "wb") as file:
        file.write(response.content)


def download_ssbd_release_zip(download_to_folder):
    """Usage
    download_ssbd_release_zip("ssbd-release")
    """
    ssbd_zip_file = f"{download_to_folder}/ssbd.zip"
    if not exists(ssbd_zip_file):
        wget(
            "https://rolandgoecke.files.wordpress.com/2019/11/ssbd-release.zip",
            ssbd_zip_file,
        )
        unzip_file(ssbd_zip_file, download_to_folder)


def process_xml(a):
    xml = read_xml(a)
    url = xml.video.url
    behaviour = xml.video.behaviours.behaviour
    behaviour = [behaviour] if not isinstance(behaviour, L) else behaviour
    time_stamps = [
        [url, stem(a), *[int(t) for t in b.time.replace("-", ":").split(":")]]
        for b in behaviour
    ]
    return time_stamps


def create_annotations(annotations_folder):
    Info(f"{len(annotations_folder.ls())} overall annotation files available")
    annotations = flatten([process_xml(a) for a in sorted(annotations_folder.ls())])
    annotations = pd.DataFrame(annotations, columns=["url", "video", "start", "end"])
    annotations["clip_duration"] = annotations["end"] - annotations["start"]
    return annotations


def fill_gaps_fn(x):
    time = x.time
    duration = int(x.duration)
    times = [clip.split(":") for clip in time.split(",")]
    classes = [x.name for _ in times]
    times, classes = update_start_ends(times, classes, duration)
    times = [":".join([str(i) for i in clip]) for clip in times[0]]
    return list(zip(times, classes))


def assign(x):
    start, end = [int(i) for i in x[0].split(":")]
    class_ = x[1].split("_")[1] if x[1] != "others" else x[1]
    return start, end, class_


def update_start_ends(start_ends, activity, video_duration):
    start_ends = [[int(start), int(end)] for start, end in start_ends]
    new_start_ends = []
    new_activity = []

    if start_ends[0][0] > 0:
        new_start_ends.append([0, start_ends[0][0]])
        new_activity.append("others")

    for i in range(len(start_ends)):
        new_start_ends.append(start_ends[i])  # Add the current activity
        new_activity.append(activity[i])

        if i < len(start_ends) - 1:
            gap_start = start_ends[i][1]
            gap_end = start_ends[i + 1][0]
            new_start_ends.append([gap_start, gap_end])
            new_activity.append("others")
    if start_ends[-1][1] < video_duration:
        new_start_ends.append([start_ends[-1][1], video_duration])
        new_activity.append("others")
    return [new_start_ends], new_activity


@cli.command()
def setup_annotations(root_folder, fill_gaps: bool = False, videos_folder: str = None):
    root_folder = P(root_folder)
    if videos_folder is not None:
        videos_folder = P(videos_folder)
    save_to = f"{root_folder}/annotations.csv"
    if exists(save_to) and not fill_gaps:
        return pd.read_csv(save_to)
    save_path = "ssbd-release/"
    download_ssbd_release_zip(download_to_folder=root_folder / save_path)

    annotations_folder = root_folder / save_path / "Annotations"
    annotations = create_annotations(annotations_folder)
    if not fill_gaps:
        annotations.to_csv(save_to, index=False)
        return

    assert exists(videos_folder), f"Videos folder is not present at `{videos_folder}`"
    available_videos = stems(videos_folder)
    annotations = annotations.query("video in @available_videos")
    durations = [
        (video, int(EncodedVideo.from_path(videos_folder / f"{video}.mp4").duration))
        for video in annotations.video.unique()
    ]
    durations = pd.DataFrame(durations, columns=["video", "duration"])
    annotations = pd.merge(annotations, durations)

    annotations["time"] = (
        annotations["start"].map(str) + ":" + annotations["end"].map(str)
    )
    frame_data = annotations.groupby("video").agg(
        {"time": lambda x: ",".join(x), "duration": max}
    )

    frame_data["activity"] = frame_data.apply(fill_gaps_fn, axis=1)
    frame_data = frame_data.explode("activity")
    frame_data[["start", "end", "label"]] = (
        frame_data["activity"].map(assign).apply(pd.Series)
    )
    frame_data = frame_data.drop(["time", "activity", "duration"], axis=1).reset_index()

    annotations = (
        pd.merge(
            annotations.drop(["start", "end", "time"], axis=1)
            .drop_duplicates()
            .reset_index(drop=True),
            frame_data,
            on="video",
        )
        .query("end <= duration")
        .reset_index(drop=True)
    )
    annotations.to_csv(save_to, index=False)
    Info(
        f"Annotations contain\n"
        f"{len(annotations[annotations['label'] != 'others'])} clips excluding 'others' category and\n"
        f"{len(annotations)} clips including 'others' category\n"
        f"for {annotations.video.nunique()} videos"
    )
    print(annotations.head())
    Info(f"saved annotations at {save_to}")
    return annotations

# Usage

```python
root = P("/mnt/347832F37832B388/ml-datasets/ssbd/")
setup_annotations(root)

# AFTER DOWNLOADING THE VIDEOS
root = P("/mnt/347832F37832B388/ml-datasets/ssbd/")
setup_annotations(
    root,
    fill_gaps=True,
    videos_folder="/mnt/347832F37832B388/ml-datasets/ssbd/ssbd-raw-videos/",
)
```


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()